In [3]:
import os
# Comprueba si el código se está ejecutando en Google Colab
try:
    import google.colab
    IN_COLAB = True
except:
    IN_COLAB = False

path_absolute = ''
if IN_COLAB:
    print("El código se está ejecutando en Google Colab.")
    from google.colab import drive

    drive.mount('/content/drive')
    path_absolute = '/content/drive/Othercomputers/Mi_portátil/TFM/WorkSpace/Models/colaborative filter'

    # Cambia al directorio de tu carpeta en Google Drive
    os.chdir(path_absolute)

    # Lista los archivos y carpetas en el directorio actual
    contenido_carpeta = os.listdir(path_absolute)
    print("Contenido de la carpeta en Google Drive:")
    print(contenido_carpeta)
else:
    print("El código se está ejecutando en un entorno local.")
    path_absolute = os.getcwd().replace("\\", "/")

datasets_path = "/datasets/"
path_absolute = path_absolute+datasets_path

El código se está ejecutando en un entorno local.


In [4]:
import pandas as pd
from scipy import sparse

In [5]:
ratings = pd.read_csv(path_absolute+"ratings.csv")
movies = pd.read_csv(path_absolute+"movies.csv")
ratings = pd.merge(movies,ratings).drop(['genres','timestamp'],axis=1)
print(ratings.shape)
ratings.head()

(97243, 4)


,movieId,title,userId,rating
0,1,Toy Story (1995),7,3.0
1,1,Toy Story (1995),9,4.0
2,1,Toy Story (1995),13,5.0
3,1,Toy Story (1995),15,2.0
4,1,Toy Story (1995),19,3.0


In [6]:
userRatings = ratings.pivot_table(index=['userId'],columns=['title'],values='rating')
userRatings.head()
print("Before: ",userRatings.shape)
userRatings = userRatings.dropna(thresh=10, axis=1).fillna(0,axis=1)
#userRatings.fillna(0, inplace=True)
print("After: ",userRatings.shape)

Before:  (671, 7062)
After:  (671, 2243)


**df.corr(method='pearson'):**

Esta función calcula la correlación entre las columnas de un DataFrame.
method='pearson' indica que se está utilizando el coeficiente de correlación de Pearson, que mide la correlación lineal entre dos variables continuas. Toma valores entre -1 y 1, donde 1 significa una correlación positiva perfecta, -1 significa una correlación negativa perfecta, y 0 significa ausencia de correlación lineal.
En el contexto del código que proporcionaste, se utiliza para calcular la correlación entre todas las columnas del DataFrame df.

In [7]:
corrMatrix = userRatings.corr(method='pearson')
corrMatrix.head(100)

title,"'burbs, The (1989)",(500) Days of Summer (2009),...And Justice for All (1979),10 Things I Hate About You (1999),101 Dalmatians (1996),101 Dalmatians (One Hundred and One Dalmatians) (1961),102 Dalmatians (2000),12 Angry Men (1957),12 Years a Slave (2013),127 Hours (2010),...,Young Guns II (1990),Zack and Miri Make a Porno (2008),Zero Dark Thirty (2012),Zero Effect (1998),Zodiac (2007),Zombieland (2009),Zoolander (2001),eXistenZ (1999),xXx (2002),¡Three Amigos! (1986)
title,,,,,,,,,,,,,,,,,,,,,
"'burbs, The (1989)",1.000000,0.049358,0.183453,0.082564,0.101948,0.162694,-0.002466,0.067309,0.020832,0.105458,...,0.297098,0.015872,0.040455,0.203206,0.082473,0.086927,0.191698,0.057206,0.099502,0.308778
(500) Days of Summer (2009),0.049358,1.000000,0.026848,0.230095,0.078582,0.040188,0.068425,0.156769,0.117532,0.380868,...,0.000182,0.172106,0.345785,-0.034352,0.279399,0.348115,0.161343,-0.029451,0.107851,0.013626
...And Justice for All (1979),0.183453,0.026848,1.000000,0.040358,0.141549,0.334189,0.005991,0.283803,-0.017748,0.069345,...,0.013836,-0.018287,0.009675,0.081699,0.036289,0.007654,0.036299,0.132239,-0.014250,0.220807
10 Things I Hate About You (1999),0.082564,0.230095,0.040358,1.000000,0.180185,0.182573,0.163474,0.065632,0.027600,0.106938,...,0.031471,0.178147,0.123387,0.121883,0.170970,0.126995,0.256138,0.101245,0.173165,0.171767
101 Dalmatians (1996),0.101948,0.078582,0.141549,0.180185,1.000000,0.353796,0.118462,0.045265,-0.030411,0.037087,...,0.098240,0.008197,0.015425,0.089189,0.053503,0.011000,0.037184,0.024336,0.023662,0.123471
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
All That Jazz (1979),0.071329,-0.006805,0.331012,-0.016868,0.083577,0.235435,-0.019459,0.154768,-0.022937,0.030181,...,0.125995,-0.023633,0.077255,0.052386,0.032157,-0.003498,0.004471,0.137846,-0.029133,0.096082
All the President's Men (1976),0.049501,0.083447,0.113703,0.155949,0.055873,0.166671,-0.019675,0.237888,-0.023192,0.056458,...,0.030931,0.128158,0.212604,0.205349,0.190511,0.080037,0.154620,0.184145,0.153382,0.212396
Almost Famous (2000),0.098950,0.172402,0.083896,0.161910,0.015288,0.109806,0.034089,0.203425,0.046136,0.110788,...,0.162815,0.139108,0.225793,0.065256,0.276196,0.172509,0.251615,0.094895,0.123510,0.129009


In [8]:
def get_similar(movie_name,rating):
    similar_ratings = corrMatrix[movie_name]*(rating-2.5)
    similar_ratings = similar_ratings.sort_values(ascending=False)
    #print(type(similar_ratings))
    return similar_ratings

In [15]:
romantic_lover = [("(500) Days of Summer (2009)",5),("Alice in Wonderland (2010)",3),
                  ("Aliens (1986)",1),("2001: A Space Odyssey (1968)",2)]
similar_movies = pd.DataFrame()
for movie,rating in romantic_lover:
    similar_movies = similar_movies.append(get_similar(movie,rating),ignore_index = True)

similar_movies.head(14)

C:\Users\jaime\AppData\Local\Temp\ipykernel_24448\3881490023.py:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  similar_movies = similar_movies.append(get_similar(movie,rating),ignore_index = True)
C:\Users\jaime\AppData\Local\Temp\ipykernel_24448\3881490023.py:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  similar_movies = similar_movies.append(get_similar(movie,rating),ignore_index = True)
C:\Users\jaime\AppData\Local\Temp\ipykernel_24448\3881490023.py:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  similar_movies = similar_movies.append(get_similar(movie,rating),ignore_index = True)
C:\Users\jaime\AppData\Local\Temp\ipykernel_24448\3881490023.py:5: FutureWarning: The frame.append method is deprecated and will be remo

title,(500) Days of Summer (2009),"Social Network, The (2010)",Up in the Air (2009),Forgetting Sarah Marshall (2008),"Hangover, The (2009)",Juno (2007),Scott Pilgrim vs. the World (2010),Stranger than Fiction (2006),Adventureland (2009),Little Miss Sunshine (2006),...,"Craft, The (1996)","Firm, The (1993)",French Kiss (1995),"Bridges of Madison County, The (1995)",Executive Decision (1996),Waterworld (1995),Disclosure (1994),Mighty Aphrodite (1995),Broken Arrow (1996),"Birdcage, The (1996)"
0,2.500000,1.221531,1.165431,1.161772,1.143526,1.125098,1.059594,1.059370,1.040743,1.039889,...,-0.147239,-0.160731,-0.163651,-0.170907,-0.170938,-0.174239,-0.179944,-0.180253,-0.191578,-0.192027
1,0.076874,0.125936,0.082362,0.049435,0.127701,0.117507,0.150894,0.096049,0.150260,0.096228,...,-0.003071,-0.016302,-0.025429,-0.022109,0.004239,0.017598,-0.023278,-0.011509,-0.017023,-0.015931
2,-0.123409,-0.167614,-0.129684,-0.217902,-0.156767,-0.107393,-0.172516,-0.150667,-0.121234,-0.069092,...,0.000718,-0.140263,-0.019289,-0.052444,-0.000728,-0.123541,0.000184,-0.111601,-0.066030,-0.116577
3,-0.065411,-0.113242,-0.040148,-0.063106,-0.083836,-0.076505,-0.109757,-0.056387,-0.084277,-0.067471,...,0.001788,-0.024276,-0.002957,-0.008093,0.007338,-0.019030,0.012001,-0.072752,0.000105,-0.032566


In [16]:
similar_movies.sum().sort_values(ascending=False).head(14)

title
(500) Days of Summer (2009)         2.388054
Up in the Air (2009)                1.077962
Social Network, The (2010)          1.066612
Juno (2007)                         1.058707
Hangover, The (2009)                1.030624
Black Swan (2010)                   1.029750
Avatar (2009)                       1.004151
Slumdog Millionaire (2008)          1.003975
Little Miss Sunshine (2006)         0.999555
Adventureland (2009)                0.985492
Green Lantern (2011)                0.976252
Stranger than Fiction (2006)        0.948365
Tangled (2010)                      0.933885
Forgetting Sarah Marshall (2008)    0.930199
dtype: float64

In [22]:
action_lover = [("Amazing Spider-Man, The (2012)",5),("Mission: Impossible III (2006)",4),
                ("Toy Story 3 (2010)",2),("2 Fast 2 Furious (Fast and the Furious 2, The) (2003)",4)]
similar_movies = pd.DataFrame()
for movie,rating in action_lover:
    similar_movies = similar_movies.append(get_similar(movie,rating),ignore_index = True)
    # similar_movies = pd.concat([similar_movies, get_similar(movie, rating)], ignore_index=True)

similar_movies.head(10)
similar_movies.sum().sort_values(ascending=False).head(14)

C:\Users\jaime\AppData\Local\Temp\ipykernel_24448\3660044565.py:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  similar_movies = similar_movies.append(get_similar(movie,rating),ignore_index = True)
C:\Users\jaime\AppData\Local\Temp\ipykernel_24448\3660044565.py:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  similar_movies = similar_movies.append(get_similar(movie,rating),ignore_index = True)
C:\Users\jaime\AppData\Local\Temp\ipykernel_24448\3660044565.py:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  similar_movies = similar_movies.append(get_similar(movie,rating),ignore_index = True)
C:\Users\jaime\AppData\Local\Temp\ipykernel_24448\3660044565.py:5: FutureWarning: The frame.append method is deprecated and will be remo

title
Amazing Spider-Man, The (2012)                           2.787758
Mission: Impossible III (2006)                           2.205062
2 Fast 2 Furious (Fast and the Furious 2, The) (2003)    2.014295
Skyfall (2012)                                           1.968146
21 Jump Street (2012)                                    1.934019
Man of Steel (2013)                                      1.877483
Oblivion (2013)                                          1.874349
Ant-Man (2015)                                           1.858115
Megamind (2010)                                          1.855477
X-Men: First Class (2011)                                1.830729
X-Men Origins: Wolverine (2009)                          1.818914
Super 8 (2011)                                           1.818343
Dredd (2012)                                             1.801364
Transporter, The (2002)                                  1.795380
dtype: float64